In [0]:
%run ./00_config

In [0]:
from pyspark.sql.functions import col, to_timestamp, to_date, current_timestamp

try:
    print("⏳ Iniciando procesamiento de la Capa Silver...")
    
    # 1. LECTURA DE CAPA BRONZE
    df_viajes = spark.table(BRONCE_TAXI_TABLE)
    
    # 2. LIMPIEZA Y NORMALIZACIÓN (Idempotencia y Particionado)
    print("🧹 Limpiando y estandarizando datos (Casteos, Filtros y Duplicados)...")
    
    df_silver = df_viajes.dropDuplicates(["hvfhs_license_num", "pickup_datetime","Dropoff_datetime", "PULocationID","DOLocationID","driver_pay","request_datetime"]) \
                         .withColumn("pickup_datetime", to_timestamp(col("pickup_datetime"))) \
                         .withColumn("dropoff_datetime", to_timestamp(col("dropoff_datetime"))) \
                         .withColumn("pickup_date", to_date(col("pickup_datetime"))) \
                         .withColumn("base_passenger_fare", col("base_passenger_fare").cast("decimal(10,2)")) \
                         .withColumn("tolls", col("tolls").cast("decimal(10,2)")) \
                         .withColumn("sales_tax", col("sales_tax").cast("decimal(10,2)")) \
                         .withColumn("congestion_surcharge", col("congestion_surcharge").cast("decimal(10,2)")) \
                         .withColumn("airport_fee", col("airport_fee").cast("decimal(10,2)")) \
                         .withColumn("tips", col("tips").cast("decimal(10,2)")) \
                         .withColumn("bcf", col("bcf").cast("decimal(10,2)")) \
                         .filter(col("PULocationID").isNotNull()) \
                         .filter(col("dropoff_datetime") > col("pickup_datetime"))

    # 3. ENRIQUECIMIENTO (JOIN con Zonas)
    print("🗺️ Enriqueciendo datos con el catálogo de zonas...")
    
    # Validación: Verificamos si la tabla de zonas existe antes de cruzar
    if spark.catalog.tableExists(BRONCE_ZONES_TABLE):
        df_zonas = spark.table(BRONCE_ZONES_TABLE)
        df_final_silver = df_silver.join(
            df_zonas, df_silver.PULocationID == df_zonas.LocationID,
            "left"
        ).select(
            df_silver["*"],
            col("Borough").alias("pickup_borough"),
            col("Zone").alias("pickup_zone")
        )
    else:
        print(f"⚠️ Tabla '{BRONCE_ZONES_TABLE}' no encontrada. Omitiendo JOIN de zonas por ahora.")
        df_final_silver = df_silver

    # 4. AUDITORÍA Y PERSISTENCIA (Idempotencia con ReplaceWhere)
    df_final_silver = df_final_silver.withColumn("silver_processing_timestamp", current_timestamp())

    print(f"💾 Guardando datos refinados (Idempotente) en la tabla: {SILVER_TAXI_TABLE}...")
    
    # Definimos la condición lógica para la sobreescritura dinámica.
    # Se deja fijo en esta parte por que sabemos que son datos de un solo mes en produccion real se dejaria variable con los ultimos 60 dia a hoy
    condicion_reemplazo = "pickup_date >= '2025-01-01' AND pickup_date <= '2025-01-31'"

    # Guardamos usando formato Delta, particionando por fecha y reemplazando solo lo necesario
    df_final_silver.write.format("delta") \
                .mode("overwrite") \
                .option("overwriteSchema", "true") \
                .option("replaceWhere", condicion_reemplazo) \
                .partitionBy("pickup_date") \
                .saveAsTable(SILVER_TAXI_TABLE)

    print("✅ Capa Silver completada con éxito.")

    # =========================================================================
    # 5. GOBIERNO DE DATOS: CONTROL DE SNAPSHOTS Y TIME TRAVEL
    # =========================================================================

    print("🔍 Historial de versiones de la tabla Silver:")
    # 1. Ver el historial de todas las operaciones (Auditoría)
    display(spark.sql(f"DESCRIBE HISTORY {SILVER_TAXI_TABLE}"))

    print("\n⏪ Consultando una versión anterior (Time Travel):")
    # 2. Leer la tabla exactamente como estaba en su Versión 0 
    df_version_0 = spark.read.format("delta").option("versionAsOf", 0).table(SILVER_TAXI_TABLE)
    display(df_version_0.limit(5))

    # 3. ROLLBACK (Desarrollo Conceptual)
    # Si se corrompen los datos, 
    # usaríamos este comando SQL para restaurar la tabla a un estado saludable:
    print("\n🔄 Comando SQL para Rollback ante fallos:")
    print(f"RESTORE TABLE {SILVER_TAXI_TABLE} TO VERSION AS OF 0;")
    # O restaurar por tiempo: RESTORE TABLE {SILVER_TAXI_TABLE} TO TIMESTAMP AS OF '2025-01-15 10:00:00';

except Exception as e:
    print(f"❌ Error en Capa Silver: {str(e)}")